# How to scrub data and make friends

In this tutorial we are going to walk through the process of building a small scale data pipeline touching on most of your project requirements along the way. I've built this with the hope of introducing you to some new ideas and concepts within Python.

- Packages - while you've been learning about the language itself we are going to look at packages, how to find them and how you can learn more about their usage. We will specifically look at reviewing docs one of the most valuable tools avaialble to you as a programmer.

- collections - this gem sits in the Python Standard Library and has a lot of awesome data structures. One that has been useful to me time again when building data applications is the namedtuple.

- String manipulations - just because there's more data in the world today doesn't mean it's ready for consumption. Even data that looks clean at first glance may hold some inconsistencies that can break your assumptions quickly. We are going to look at ways to handle cleaning strings using python string type methods and regex

- Iteratating - Python has lots of tools for iterating over collections and strings. We will look at some of the more well known and widely used options such as the for loop and list comprehension as well as showing the itertools Standard Library package.

- Storing data - as part of our data pipeline we probably want to save data for later use or sharing with others. While various file options exist those are prone to error both on read and write. We will make use of SQLite being available as part of the Python Standard Library to store the data, and then show how it can be used to make some analysis easier with simple queries.

## Getting Started
To begin let's' import all the packages we know we will need. Later on we may decide we need new packages in which can we can import them inline. Later on once we have a more finished notebook we could move them back to the top to make it easy to see what packages are needed to run this notebook.

In [1]:
import sqlite3
import re
import requests
from math import pi
from itertools import chain
from collections import namedtuple
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, output_file, show
from bokeh.models import HoverTool, FuncTickFormatter, FixedTicker, ColumnDataSource

## Project

For this project I wanted to pull some data from a remote api. I found a coffee dataset that looked interesting on data.world and figured that would be a good source for the purposes of this tutorial.

This tutorial will walk through using the requests library to talk with the data.world API pulling a remote datasource into a Python object where we will then proceed to move the data through a variety of structures cleaning the data up and analyzing some of it's characteristics. Once we have the data in a clean format we will write it to a database as a check point from which we can pick back up or provide the data to other users. 

We will then examine how SQL Queries can make certain operations fast and straight forward for aggregating or manipulating datasets and how to load data back from our database into a new Python object. 

After that we will review manipulating the data once more to put it into formats easy to visualize with Bokeh and provide a quick analysis of some findings from the Austion Coffee Yelp Reviews dataset.

### Datasource
https://data.world/rdowns26/austin-coffee-yelp-reviews

### Data.world API
https://data.world/about/ signup

https://apidocs.data.world/v0/data-world-for-developers/api-quickstart

https://data.world/settings/advanced get your Auth Key

## Getting the Data

Let's make use of the awesome Requests library for our HTTP call. http://docs.python-requests.org/en/master/


Per the data.world API docs we need to pass our Authorization key and a Content-type in the headers. We can setup and pass custom headers easily with requests in the GET call by creating and passing a Python dictionary. http://docs.python-requests.org/en/master/user/quickstart/#custom-headers

Checkout the coffee_reviews object inspecting the type and available functions/attributes. We can extract the text and begin reviewing the data.

In [2]:
token = ''
header = headers = {'Authorization': f'Bearer {token}','Content-type': 'application/json'}

coffee_reviews = requests.get('https://api.data.world/v0/file_download/rdowns26/austin-coffee-yelp-reviews/raw_yelp_review_data.csv', 
                             headers=headers)

#dir(coffee_reviews) uncomment this, but know that the list is long
#type(coffee_reviews)
raw_text = coffee_reviews.text

## Parsing the data

What do you notice about the text we have received? Is there a common row delimiter? What about a common column delimiter we can use?

https://stackoverflow.com/questions/3091524/what-are-carriage-return-linefeed-and-form-feed

In [3]:
review_list = raw_text.split('\r')
review_list[3]

'The Factory - Cafe With a Soul ," 11/30/2016 1 check-in Listed in Brunch Spots I loved the eclectic and homey plush decor and who doesn\'t want to swing and drink their coffee?  I would categorize this as a pricier coffee place but, to me it was worth it.  After Thanksgiving nobody wants to make brunch so we headed out in search of chicken and waffles.  There is ample parking in the private lot and  ordering is done through I-Pads.  Pick a seat and they will call your name when your items are ready.  Be patient because some of the coffee drinks take longer than others.  I had the cold brew coffee, chicken and waffles and we shared a pop+art!  The po+art was strawberry; light and fluffy on the outside and gooey sweet filling on the inside.  The chicken and waffles wasn\'t the standard so expect different, complex and interesting flavors from the accompanying slaw and sauce.  I thoroughly enjoyed it but, the waffle was a bit bland and maybe could have used a bit of honey to spruce it up

## Introducing some new Python concepts

A lot of different python structures and functions are about to be used as we attempt to clean and parse the data we have received. You may recognize some or you might not recognize any of the functions and structures below. That's ok as you come across new concepts try them in your REPL, ask questions and definitely read the documentation. The format and language of the docs may seem weird even alien at first, but over time they will begin to make more sense and be an invaluable tool for you as a programmer.

- namedtuple - https://docs.python.org/3.6/library/collections.html
- list comprehension - https://docs.python.org/3.6/tutorial/datastructures.html#list-comprehensions
- strings - https://docs.python.org/3.6/library/string.html
- split - https://docs.python.org/3.6/library/stdtypes.html#str.split
- join - https://docs.python.org/3.6/library/stdtypes.html#str.join
- map - https://docs.python.org/3.6/library/functions.html#map
- lambda - https://docs.python.org/3.6/reference/expressions.html#lambda


In [4]:
formatted_reviews = [r.split(',') for r in review_list]
review = namedtuple('review', 'shop_name, review, stars')
#We can drop the first row of header data and remove leading/trailing whitespace from the strings
reviews = [review(r[0].strip(), r[1].strip(), r[2].strip()) for r in formatted_reviews[1:]]
formatted_reviews[5]

['The Factory - Cafe With a Soul ',
 '" 12/3/2016 1 check-in They are located within the Northcross mall shopping center facing east toward Burnet Rd with plenty of parking. I loved their computerized user friendly ordering system',
 ' it made it easy me to pick & choose all the items I wanted to try. I ordered pop tart',
 ' chicken & waffles',
 ' strawberry waffles (photos uploaded). This place is ideal for ordering a few dishes to share with your group. Seating is limited since this place is fairly new with lots of visitors. Arrive early and be prepared to wait a bit if you with a large grop. Store front facing Burnet Rd See all photos from Michelle A. for The Factory - Cafe With a Soul "',
 ' 4.0 star rating ']

## Wait what happened?

This is a learning tutorial things shouldn't go wrong right? Well in a few places I've left my mistakes in to show where some of my assumptions didn't hold up and what changes had to be made. Ok, so it turns out even though the first few rows of the review dataset seemed to split on commas, we have later rows where users placed commas in their review causing our lists to have more indices available then expected. Because of that when we wanted to transform each row into a namedtuple for easier interaction later we couldn't do that because of index problems. Looking back at the strings it looks like we could potentially make use of ", or ," to split the review into the columns we want. Let's try it.

In [5]:
try:
    delimited_reviews = list(map(lambda x: x.replace(',"','",'), review_list))
    formatted_reviews = [r.split('",') for r in delimited_reviews]
    reviews = [review(r[0].strip(), r[1].strip(), r[2].strip()) for r in formatted_reviews[1:]]
except IndexError as e:
    print(e)
    pass

list index out of range


Ok, so that didn't work either, but the dataset said it was delimited by commas. We know there are three columns (store name, review text, rating) that we want so is there something we might be able to do. Finding commas plus another parse step?

We have strings, we can make those strings into list, list have indices, we know that index 0 is the store name and the last index is the rating. Ah, knowing that let's trying splitting up each string into the store name, the rating, and then all the other indices are our review text. We can then take and flatten those indices into one row and now we have our store name, rating and review. 

https://softwareengineering.stackexchange.com/questions/70743/what-does-flatten-mean

In [6]:
stars = [r.split(',')[-1] for r in review_list[1:]]
shops = [r.split(',')[0] for r in review_list[1:]]
#Put all the data in between the first and last index of the string split into one list per row
reviews = [r.split(',')[1:-1] for r in review_list[1:]]
#Flatten the list structure so each row is one string containing the review
flat_reviews = [''.join(map(str, r)) for r in reviews]
flat_reviews[0]

'" 11/25/2016 1 check-in Love love loved the atmosphere! Every corner of the coffee shop had its own style and there were swings!!! I ordered the matcha latte and it was muy fantastico! Ordering and getting my drink were pretty streamlined. I ordered on an iPad which included all beverage selections that ranged from coffee to wine desired level of sweetness and a checkout system. I got my latte within minutes!  I was hoping for a typical heart or feather on my latte but found myself listing out all the possibilities of what the art may be. Any ideas? "'

## And now we have three

But now we have three individual list which means any time we want to access a full row of information we have to look up the data at the specific index in each list. It would be a lot easier if we could put all of this back together into a list of collections (in this case tuples instead of a list of list) where there is one list with all the reviews, and then in each sub list we have the store name, review and rating available for lookup.

Enter the handy zip function https://docs.python.org/3.6/library/functions.html#zip

In [7]:
formatted_reviews = list(zip(shops, flat_reviews, stars))
type(formatted_reviews[0])
formatted_reviews[0]

('The Factory - Cafe With a Soul ',
 '" 11/25/2016 1 check-in Love love loved the atmosphere! Every corner of the coffee shop had its own style and there were swings!!! I ordered the matcha latte and it was muy fantastico! Ordering and getting my drink were pretty streamlined. I ordered on an iPad which included all beverage selections that ranged from coffee to wine desired level of sweetness and a checkout system. I got my latte within minutes!  I was hoping for a typical heart or feather on my latte but found myself listing out all the possibilities of what the art may be. Any ideas? "',
 ' 5.0 star rating ')

Ok, so now we have our rows properly formatted and easily indexable. But sometimes we have data sets with more than three columns. They might have 10, 20 or more. When that happens it's hard to remember what data is at a given index. Instead we can use the named tuple we defined earlier and refer to field/column names instead of specific indices.

In [8]:
reviews = [review(r[0].strip(), r[1].strip(), r[2].strip()) for r in formatted_reviews]
print(f"Type: {type(reviews[0])}\nStore: {reviews[0].shop_name} Reviews: {reviews[0].stars}\nReview: {reviews[0].review}")

Type: <class '__main__.review'>
Store: The Factory - Cafe With a Soul Reviews: 5.0 star rating
Review: " 11/25/2016 1 check-in Love love loved the atmosphere! Every corner of the coffee shop had its own style and there were swings!!! I ordered the matcha latte and it was muy fantastico! Ordering and getting my drink were pretty streamlined. I ordered on an iPad which included all beverage selections that ranged from coffee to wine desired level of sweetness and a checkout system. I got my latte within minutes!  I was hoping for a typical heart or feather on my latte but found myself listing out all the possibilities of what the art may be. Any ideas? "


In [9]:
review_count = len(reviews)
#If you want to get a unique set of data move into a set for instance shop names
#https://docs.python.org/3.6/library/stdtypes.html?highlight=set#set
shops = set([shop.shop_name for shop in reviews])
print(f"Total Reviews: {review_count}\n,Unique Shops: {shops}")

Total Reviews: 7616
,Unique Shops: {'Thunderbird Coffee', 'Caffe Medici', 'La Tazza Fresca', 'Flitch Coffee', 'Juan Pelota', 'Friends & Neighbors', 'The Steeping Room', 'CafÃ© CrÃ¨me', 'The Hideout Coffeehouse', 'Live Oak Market', 'TOMS', 'Venezia Italian Gelato', 'Elixer Coffee', 'Quackâ\x80\x99s 43rd Street Bakery', 'Houndstooth Coffee', 'Stinsonâ\x80\x99s', 'The Factory - Cafe With a Soul', 'Trianon Coffee', 'Vintage Heart Coffee', 'Bennu Coffee', 'The Marvelous Vintage Tea Party Co.', 'Tea Haus', 'Monkey Nest Coffee', 'Fat Cats Organic Coffee And Dessert', '"Strange Brew', 'Tuscany At 360', 'Genuine Joe', 'My Sweet Austin', 'Holy Grounds', 'Altaâ\x80\x99s Cafe', 'Apanas Coffee & Beer', 'Radio Coffee & Beer', 'Fleet Coffee', 'Joâ\x80\x99s Coffee', 'Lucky Lab Coffee', 'Mozartâ\x80\x99s Coffee Roasters', 'The Buzz Mill', 'CaffÃ© Medici', 'Maryâ\x80\x99s Cafe', 'Andersonâ\x80\x99s Coffee Company', 'Dolce Neve', 'Summer Moon Coffee Bar', 'Emerald Tavern Games & Cafe', 'Summermoon Coffee

## Tidying up

When we printed the names above it looks like we have characters that are not being parsed right. Since the data.world page doesn't provide any encoding information we could trying changing the enoding on the data set to test if we can find the right encoding. This is error prone and has some dangers. If you want a good read check out https://nedbatchelder.com/text/unipain.html.

For now we are going to make use of regular expressions to remove the characters that are not parsing correctly. 

- https://docs.python.org/3/library/re.html
- https://regexr.com/
- http://man7.org/linux/man-pages/man7/regex.7.html

In [10]:
scrubbed = [re.sub(r'[^\x00-\x7F]+',' ', shop) for shop in shops]
print(f"Total Reviews: {review_count}\n Unique Shops: {scrubbed}")

Total Reviews: 7616
 Unique Shops: ['Thunderbird Coffee', 'Caffe Medici', 'La Tazza Fresca', 'Flitch Coffee', 'Juan Pelota', 'Friends & Neighbors', 'The Steeping Room', 'Caf  Cr me', 'The Hideout Coffeehouse', 'Live Oak Market', 'TOMS', 'Venezia Italian Gelato', 'Elixer Coffee', 'Quack s 43rd Street Bakery', 'Houndstooth Coffee', 'Stinson s', 'The Factory - Cafe With a Soul', 'Trianon Coffee', 'Vintage Heart Coffee', 'Bennu Coffee', 'The Marvelous Vintage Tea Party Co.', 'Tea Haus', 'Monkey Nest Coffee', 'Fat Cats Organic Coffee And Dessert', '"Strange Brew', 'Tuscany At 360', 'Genuine Joe', 'My Sweet Austin', 'Holy Grounds', 'Alta s Cafe', 'Apanas Coffee & Beer', 'Radio Coffee & Beer', 'Fleet Coffee', 'Jo s Coffee', 'Lucky Lab Coffee', 'Mozart s Coffee Roasters', 'The Buzz Mill', 'Caff  Medici', 'Mary s Cafe', 'Anderson s Coffee Company', 'Dolce Neve', 'Summer Moon Coffee Bar', 'Emerald Tavern Games & Cafe', 'Summermoon Coffee Bar', 'Legend Coffee', 'Ma ana Coffee & Juice', 'Dominican

## Checkpoint

Ok, we've got our data in a nice format. We've done a lot of work to reach this point. Let's make sure we can get back to this data easily by storing it in a database. 

Out of the box Python comes with SQLite bundled into the standard library. This is a really handy tool to know about and make use of. For more information about sqlite and the python module checkout:

- https://sqlite.org/
- https://docs.python.org/3.6/library/sqlite3.html

In [11]:
conn = sqlite3.connect('austin_reviews.db')
cur = conn.cursor() 
cur.execute('''CREATE TABLE reviews (store TEXT, review TEXT, raw_score TEXT)''')

In [12]:
for review in reviews:
  cur.execute('INSERT INTO reviews VALUES (?,?,?)', review)
conn.commit()

In [13]:
results = cur.execute('SELECT raw_score FROM reviews LIMIT 5;')
try:
    first_row = next(results)
    for row in chain((first_row,), results):
        print(row)
except StopIteration as e:
    pass

('5.0 star rating',)
('4.0 star rating',)
('4.0 star rating',)
('2.0 star rating',)
('4.0 star rating',)


## We missed something

What if I want to quickly calculate an average score from the reviews. Turns out our rating field has the string "star rating" embedded in it. Lets rip that out and just keep the actual score so that we can perform different calculations with the rating without having to remove the "star rating" string each time.

In [14]:
austin_scores = [re.findall(r'\d+\.\d+', review.stars) for review in reviews]
flattened_scores = list(chain.from_iterable(austin_scores))
flattened_scores = [float(score) for score in flattened_scores]
average_austin = sum(flattened_scores) / len(austin_scores)
print(f"Total review scores in Austion {len(austin_scores)}\nAverage score for Austion: {average_austin}")

Total review scores in Austion 7616
Average score for Austion: 4.169117647058823


## That's nice, but.....

But now we have a field that isn't part of our named tuple, and isn't in our database. Let's fix both of those problems. First we need to defined a new named tuple and then iterate over our old tuple and rating to move it into a new structure with the new field.

Once we've done that we also want to store this in our database for long term use and sharing. We can update our table with an additional field, truncate the table data (remove all the old data) and load in the reformatted dataset.

In [15]:
reviews_final = list(zip(reviews, flattened_scores))
review_with_rating = namedtuple('review', 'shop_name, review, stars, rating')
send_to_sql = []
for r in reviews_final:
    send_to_sql.append(review_with_rating(r[0].shop_name, r[0].review, r[0].stars, r[1]))

In [16]:
#cur = conn.cursor() 
cur.execute('''ALTER TABLE reviews ADD rating REAL;''')
conn.commit()
cur.execute('''DELETE FROM reviews;''')
conn.commit()

In [17]:
for review in send_to_sql:
    cur.execute("INSERT INTO reviews VALUES (?,?,?,?)", review)
conn.commit()

In [18]:
result = cur.execute('SELECT SUM(rating) FROM reviews')
try:
    first_row = next(result)
    for row in chain((first_row,), result):
        print(row)
except StopIteration as e:
    pass

(31752.0,)


In [19]:
cur.execute('''CREATE INDEX store_idx on reviews(store);''')
conn.commit()
avg_store_review = cur.execute('SELECT store, AVG(rating) as rating FROM reviews GROUP BY store ORDER BY rating DESC')
avg_scores = []
try:
    first_row = next(avg_store_review)
    for row in chain((first_row,), avg_store_review):
        avg_scores.append(row)
except StopIteration as e:
    pass
avg_scores[1]

('The Marvelous Vintage Tea Party Co.', 5.0)

## Let's visualize

Ok, so far we have pulled data from the data.world api, converted it into a python collection that is easy for us to work with and manipulate, and loaded it into a database for easy sharing and recall at a later time.

Now let's visualize the data using Bokeh. Let's do something simple and show the top 10 and bottom 10 reviews on a simple vertical bar chart. 

### Warning Bokeh had a breaking change in their API in March of 2017 release 0.12.5
That means many stackoverflow post and older documentation that you might find will not work on newer releases of Bokeh. Because of that make sure to pay attention to dates and Bokeh versions in post you come across. Specifically the bokeh.charts module was frequently used and has been removed. 

For more information read the below:
- https://bokeh.pydata.org/en/latest/docs/releases/0.12.5.html
- https://bokeh.pydata.org/en/latest/docs/user_guide.html

In [20]:
type(avg_scores)
#Pull the rating and store names into their own list to pass to bokeh for the different axis and plot values.
ratings = [row[1] for row in avg_scores]
stores = [row[0] for row in avg_scores]

#Set to open charts in the notebook. 
output_notebook()

#Every plot starts with a figure object that defines some generic information relative to the plots display
p = figure(x_range=stores[:10], plot_height=400, plot_width=800, title="Top 10 Average Scores",
           toolbar_location=None, tools="")

#With a figure defined we then declar the type of chart (vbar) and then pass it the data to plot
p.vbar(x=stores[:10], top=ratings[:10], width=0.9, color='#4BC246')
#We can also manipulate attribute values on our figure from above after declaration 
p.xaxis.major_label_orientation = pi/4
p.yaxis.major_label_orientation = "horizontal"
p.xgrid.grid_line_color = None
p.y_range.start = 0
#Once we have provided the data and any formatting information that's needed we can call the show function for the specific figure
show(p)

Loading BokehJS ...

## Now with more Pandas

Ok, we've used raw python data structures and objects up to this point. There is a library in Python called Pandas that you will definitely come across as it provides some nice functionality for interacting with tabular data from various sources. Pandas provides a lot of extra functionality out of the box such as interacting with databases, flat files, web scraping etc. It's a powerful tool, but use with care as it is a memory expensive tool.

- https://pandas.pydata.org/

For the bottom 10 plot let's define a dataframe (this is like an in memory table of data) to use with Bokeh and the Bokeh ColumnDataSource. Using Pandas and the Bokeh ColumnDataSource gives us access to some extra functionality such as hover over tool tips that can be a nice addition to our visualization of the data.

- https://bokeh.pydata.org/en/latest/docs/reference/models/sources.html
- https://bokeh.pydata.org/en/latest/docs/user_guide/tools.html
- https://bokeh.pydata.org/en/latest/docs/user_guide/tools.html#hovertool

In [21]:
#Import pandas now since it's a heavy library
import pandas as pd


df = pd.DataFrame(avg_scores, columns=['name', 'rating'])
df.head(3)
bottom_ten = df[-10:]
bottom_ten.index
#We need to reuse the index since it's maintained from the original dataframe, but Bokeh will use the index values
#to associate store names with points along the x axis of our plot.
bottom_ten = bottom_ten.reset_index(drop=True)
bottom_ten

,name,rating
0,Teo,3.880000
1,CuvÃ©e Coffee Bar,3.820000
2,Halcyon,3.820000
3,Tuscany At 360,3.818182
4,Epoch Coffee,3.812500
5,Stinsonâs,3.810000
6,Kick Butt Coffee Music & Booze,3.720000
7,Pacha,3.690000
8,"""Strange Brew",3.600000
9,Once Over Coffee Bar,3.370000


In [22]:
source = ColumnDataSource(bottom_ten)
source
source.column_names

['name', 'rating', 'index']

In [23]:
hover = HoverTool(
        tooltips=[
            ("Average Rating: ", "@rating"),
        ]
    )

rating_plot = figure(x_range=stores[-10:], y_range=(0,5), height = 400, width = 600, tools=[hover],
                    title="Bottom Ten Average Reviews")
rating_plot.vbar(x='name', top='rating', width=0.5, color='#C32B2B', source=source)
rating_plot.xaxis.axis_label = "Coffee Shop"
rating_plot.yaxis.axis_label = "Rating Scale"
rating_plot.xaxis.major_label_orientation = pi/4
rating_plot.xgrid.grid_line_color = None
show(rating_plot)

## Just for fun

The example below is an extra example showing a different plot available in bokeh. For this dataset I don't think that a scatter plot gives us much extra information. Different types of plots/charts are better at visualizing and presenting different types of data. One thing the below example does show is how to provide custom text for your axis tick labels. The enumerate function manipulates the data into a format that's easy to use with python string formatting which can then be passed to the Bokeh FuncTickFormatter for custom labels.

- https://bokeh.pydata.org/en/latest/docs/reference/models/formatters.html\
- https://docs.python.org/3.6/library/functions.html#enumerate
- https://plotlyblog.tumblr.com/post/118355223592/how-to-analyze-data-eight-useful-ways-you-can

In [24]:
label_dict = {}
for i, s in enumerate(bottom_ten.name):
    label_dict[i] = s
label_dict

circle_plot = figure(height = 400, width = 600, tools=[hover])
circle_plot.circle("index", "rating", source=source, size=20, color='#C32B2B')
circle_plot.y_range.start = 0
circle_plot.y_range.end = 5
circle_plot.xaxis.major_label_orientation = pi/4
circle_plot.xaxis[0].ticker.desired_num_ticks = 10
circle_plot.xaxis.formatter = FuncTickFormatter(code="""
    var labels = {};
    return labels[tick];
""".format(label_dict))

show(circle_plot)

## Initial Analysis

So roughly based on the data that we have looked at I can make one of a few assumptions:

1. Austin has pretty awesome coffee no matter which coffee shop you go to
2. These yelp reviews are biased and people are not reviewing bad experiences
3. Bad coffee shops in Austion don't have many users that make use of Yelp
4. Average reviews are not a good way to look at this data, I'm not looking at review count per store, lowest vs highest, etc. instead I've just taken a raw average. I may be missing something by taking such a simplistic approach.
5. I'm sure you will think of something I haven't listed above.

Now I can go back and begin writing more queries looking at each review potentially or trying to find patterns. Maybe I identify an extra data source I should load in such as coordinate data and map the locations of the coffee shops to see if I'm only getting reviews for part of Austin. Who knows what could be found.

## What's next

First things first identify a dataset that you want to connect to and analyze! There's a lot of data out there just waiting for somebody to make use of it. If you don't know where to start consider checking out:

- https://data.world/
- https://data.louisvilleky.gov/
- https://apilist.fun/ 
- https://www.kaggle.com/datasets
- https://duckduckgo.com/?q=opendata+portal&t=ffab&ia=web

Looking at our code above there is a lot we could make better. First we haven't defined any functions instead our code is almost like a stream of conciousness. That's ok for exploring ideas and concepts, but doesn't lend itself to long term code maintenance. A lot of the above should be pulled into python scripts where functions and classes are created, variables have better/more meaningful names and the code has test to help keep it maintained. Along those lines we haven't mentioned PEP8, but it's something to check out. It is the standard style guide for Python code and defines ways code should be laid out to make it easier for others to pick up review and change your code at a later date. We touched on a lot of concepts spread out across the standard library. The standard library has many more gems to explore. We touched on a lot of different ways to iterate over data with various control flow tools. There are others we didn't use such as the while loop, generators or options from the functools module. Finally we briefly used a few external python libraries such as requests, pandas and bokeh. There's a whole world of python libraries out there to explore and use to help solve new problems. Finally just keep learning, building and having fun writing python.

- https://www.python.org/dev/peps/pep-0008/
- https://pypi.org/
- https://docs.python.org/3/tutorial/controlflow.html
- https://wiki.python.org/moin/Iterator
- https://docs.python.org/3.6/library/functools.html
- https://docs.pytest.org/en/latest/